In [24]:
import pyodbc 
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("./PlayerSeason.2024.csv")

print(df.columns)
print(df.shape)

# TODO: https://www.dropbox.com/scl/fi/hy0sxw6gaai7ghemrshi8/lahman_1871-2023_csv.7z?dl=0&e=1&file_subpath=%2Flahman_1871-2023_csv&rlkey=edw1u63zzxg48gvpcmr3qpnhz

Index(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',
       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',
       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',
       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',
       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',
       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',
       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',
       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',
       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',
       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',
       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',
       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',
       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',
       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',
       'FantasyPointsDraftKings', 'Weight

In [ ]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = ''

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [26]:
df = df.dropna(subset=['TeamID'])
df = df.dropna(subset=['PlayerID'])

print(df.shape)

(5365, 66)


In [27]:
for index, row in df.iterrows():
		playerID = None
		teamID = None
		
		cursor.execute("SELECT PlayerID FROM MLB.Players WHERE ExternalPlayerID = ?", row.PlayerID)
		player = cursor.fetchone()
		if (player is not None):
			playerID = player[0]

		cursor.execute("SELECT TeamID FROM MLB.Teams WHERE ExternalTeamID = ?", row.TeamID)
		team = cursor.fetchone()
		if (team is not None):
			teamID = team[0]

		if (playerID is None or teamID is None):
			continue

		cursor.execute("""
			INSERT INTO MLB.Batting
			(
					PlayerID, TeamID, Season, SeasonType, Games, Starts, AtBats, Runs, Hits, Singles,
					Doubles, Triples, HomeRuns, RunsBattedIn, BattingAverage, Outs, Strikeouts,
					Walks, HitByPitch, Steals, CaughtStealing, OBP, Slug, OBPPlus
			)
			VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
			""",
			playerID, teamID, row.Season, row.SeasonType, row.Games, row.Started, row.AtBats, row.Runs, row.Hits, row.Singles,
			row.Doubles, row.Triples, row.HomeRuns, row.RunsBattedIn, row.BattingAverage, row.Outs, row.Strikeouts,
			row.Walks, row.HitByPitch, row.StolenBases, row.CaughtStealing, row.OnBasePercentage, row.SluggingPercentage, row.OnBasePlusSlugging)
		
		# Only insert Pitchers
		if (row.PositionCategory == 'P'):
			cursor.execute("""
				INSERT INTO MLB.Pitching
				(
						PlayerID, TeamID, Season, SeasonType, Games, Starts, Wins, Losses, Saves, InningsPitched,
						ERA, Runs, Hits, HomeRuns, Strikeouts, StrikeoutsPerNineInnings, Walks, WalksPerNineInnings,
						PitchesThrown, WHIP, BattingAverageAgainst, OBP, Slug, OBPPlus, Shutouts
				)
				VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
				""",
				playerID, teamID, row.Season, row.SeasonType, row.Games, row.Started, row.Wins, row.Losses, row.Saves, row.InningsPitchedDecimal,
				row.EarnedRunAverage, row.PitchingRuns, row.PitchingHits, row.PitchingHomeRuns, row.PitchingStrikeouts, row.PitchingStrikeoutsPerNineInnings, row.PitchingWalks, row.PitchingWalksPerNineInnings,
				row.PitchesThrown, row.WalksHitsPerInningsPitched, row.PitchingBattingAverageAgainst, row.PitchingOnBasePercentage, row.PitchingSluggingPercentage, row.PitchingOnBasePlusSlugging, row.PitchingShutOuts)

cnxn.commit()
cursor.close()

"""
Index(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',
       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',
       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',
       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',
       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',
       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',
       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',
       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',
       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',
       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',
       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',
       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',
       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',
       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',
       'FantasyPointsDraftKings', 'WeightedOnBasePercentage',
       'PitchingCompleteGames', 'PitchingShutOuts', 'PitchingOnBasePercentage',
       'PitchingSluggingPercentage', 'PitchingOnBasePlusSlugging',
       'PitchingStrikeoutsPerNineInnings', 'PitchingWalksPerNineInnings',
       'PitchingWeightedOnBasePercentage', 'FantasyPointsBatting',
       'FantasyPointsPitching'],
"""

"\nIndex(['StatID', 'TeamID', 'PlayerID', 'SeasonType', 'Season', 'Name', 'Team',\n       'Position', 'PositionCategory', 'Started', 'AverageDraftPosition',\n       'AuctionValue', 'Games', 'FantasyPoints', 'AtBats', 'Runs', 'Hits',\n       'Singles', 'Doubles', 'Triples', 'HomeRuns', 'RunsBattedIn',\n       'BattingAverage', 'Outs', 'Strikeouts', 'Walks', 'HitByPitch',\n       'Sacrifices', 'SacrificeFlies', 'GroundIntoDoublePlay', 'StolenBases',\n       'CaughtStealing', 'OnBasePercentage', 'SluggingPercentage',\n       'OnBasePlusSlugging', 'Wins', 'Losses', 'Saves',\n       'InningsPitchedDecimal', 'TotalOutsPitched', 'InningsPitchedFull',\n       'InningsPitchedOuts', 'EarnedRunAverage', 'PitchingHits',\n       'PitchingRuns', 'PitchingEarnedRuns', 'PitchingWalks',\n       'PitchingStrikeouts', 'PitchingHomeRuns', 'PitchesThrown',\n       'PitchesThrownStrikes', 'WalksHitsPerInningsPitched',\n       'PitchingBattingAverageAgainst', 'FantasyPointsFanDuel',\n       'FantasyPointsDra